In [1]:
import numpy as np
import pandas as pd

In [22]:
#Initializ with centroid data and the corresponding uncertainty generated from Feature Extraction
STAR_CENTROIDS = np.array([[-3,4], [8,-6], [10,10], [5,5]]) 
STAR_CENTROIDS_UNCERTAINTY = np.array([[0.06], [0.004], [0.001], [0.002]])

#Initialize 
CATALOGUE = pd.read_csv("Modified Star Catalogue.csv")
PROCESSED_CATALOGUE= pd.read_csv('Processed_Catalogue.csv')

In [23]:
def cos(row):
    return np.cos(np.radians(row['Ang_Distance']))

REFERENCE  = pd.DataFrame(columns=['Star_ID1', 'Star_ID2', 'Ang_Distance'])
REFERENCE['Star_ID1'], REFERENCE['Star_ID2'] = PROCESSED_CATALOGUE['Star_ID1'], PROCESSED_CATALOGUE['Star_ID2']
REFERENCE['Ang_Distance'] = PROCESSED_CATALOGUE.apply(cos, axis = 1)
REFERENCE.sort_values('Ang_Distance' ,ascending=True, inplace=True)

REFERENCE.head()

,Star_ID1,Star_ID2,Ang_Distance
925,23440,80559,-0.999995
2832,23440,57018,-0.999955
2537,23440,23564,-0.999867
2502,23440,1403,-0.999534
2242,23440,68540,-0.999342


In [94]:
VOTING_DF = pd.DataFrame(columns=['Star_ID', 'Votes'])
VOTING_DF['Star_ID'] = CATALOGUE['StarID']
VOTING_DF.sort_values('Star_ID', inplace = True, ascending = True)
VOTING_DF['Votes'].values[:] = 0
VOTING_DF.head()

,Star_ID,Votes
3614,64,0
2937,77,0
1237,91,0
3128,93,0
1881,107,0


In [25]:
STAR_CENTROIDS

array([[-3,  4],
       [ 8, -6],
       [10, 10],
       [ 5,  5]])

In [26]:
STAR_CENTROIDS_UNCERTAINTY

array([[0.06 ],
       [0.004],
       [0.001],
       [0.002]])

In [27]:
def starVectorTransform(centroid, focal_length=10):
    '''
    Generates the unit 3D vectors from given 2D centroids of stars on the 
    image frame with the focal point as the origin
    
    <Formula> - CubeStar Doc - Appendix B
    '''
    x, y = centroid
    
    temp = np.power(((x/focal_length)**2 + (y/focal_length)**2 + 1), -0.5)
    ux = (x/focal_length)
    uy = (y/focal_length)
    uz = 1
    return np.array([ux, uy, uz])*temp

STAR_VECTORS = np.apply_along_axis(starVectorTransform, 1, STAR_CENTROIDS, focal_length=10 )
STAR_VECTORS

array([[-0.26832816,  0.35777088,  0.89442719],
       [ 0.56568542, -0.42426407,  0.70710678],
       [ 0.57735027,  0.57735027,  0.57735027],
       [ 0.40824829,  0.40824829,  0.81649658]])

In [28]:
def vectorAngularDistance(vect1, vect2):
    '''
    Returns the angular distance [cos(theta)] between two unit vectors seperated by an angle theta
    '''
    return np.sum(vect1*vect2)

def uncertaintyAngularDistance(u1, u2):
    '''
    Assumes that the uncertainty is a simple addition
    '''
    return u1 + u2

In [29]:
#Generating square symmetric matrix of order (row X row), where row is the number of stars in the STAR_CENTROID array
#Each element corresponds to the angular distance [cos(theta)] between vectors Star_i & Star_j separated by angle theta
#Thus the diagonal elements will always be one

rows, columns = STAR_CENTROIDS.shape
ANGULAR_DISTANCE = np.zeros((rows,rows))
for i in range(rows):
    for j in range(i, rows):
        dist_ij = vectorAngularDistance(STAR_VECTORS[i], STAR_VECTORS[j])
        ANGULAR_DISTANCE[i,j] = ANGULAR_DISTANCE[j,i] = dist_ij

#Symmetric Matrix
ANGULAR_DISTANCE == ANGULAR_DISTANCE.T

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [30]:
ANGULAR_DISTANCE

array([[1.        , 0.32887688, 0.56803756, 0.76681158],
       [0.32887688, 1.        , 0.48989795, 0.6350853 ],
       [0.56803756, 0.48989795, 1.        , 0.94280904],
       [0.76681158, 0.6350853 , 0.94280904, 1.        ]])

In [148]:
VOTING_DF['Votes'].values[:] = 0
for i in range(rows):
    for j in range(i+1, rows):
        dist_ij = ANGULAR_DISTANCE[i,j]
        uncert_ij = uncertaintyAngularDistance(STAR_CENTROIDS_UNCERTAINTY[i], STAR_CENTROIDS_UNCERTAINTY[j])
        r_ij = [dist_ij - uncert_ij, dist_ij + uncert_ij]
        
        temp_df = REFERENCE[(REFERENCE['Ang_Distance']>r_ij[0][0]) & (REFERENCE['Ang_Distance']<r_ij[1][0])]
        
        for k in range(temp_df.shape[0]):
            s1, s2, ad = temp_df.iloc[k]
            VOTING_DF.loc[VOTING_DF['Star_ID'] == s1, 'Votes'] = VOTING_DF.loc[VOTING_DF['Star_ID'] == s1, 'Votes'] + 1
            VOTING_DF.loc[VOTING_DF['Star_ID'] == s2, 'Votes'] = VOTING_DF.loc[VOTING_DF['Star_ID'] == s2, 'Votes'] + 1

VOTING_DF.sort_values('Votes', inplace = True, ascending=False)
VOTING_DF.head(10)

,Star_ID,Votes
0,23440,993
1782,35673,1
2919,37766,1
3066,35830,1
1631,69629,1
4888,37428,1
3768,37400,1
4932,36949,1
2305,36924,1
4085,36891,1
